En este archivo .ipynb voy a trabajar en la creación y desarrollo del conjunto de datos necesario para crear la pestaña en el dashboard correspondiente a los KPI.
Tengo 2 KPI, y en este archivo me voy a centrar en la creación del segundo, la consigna del mismo es la siguiente:
- Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.
Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100